In [ ]:
from signals_visualization import upload_and_draw
from signals_classification import classify_diff_intervals
%matplotlib inline

In [ ]:
all_electrodes = ['F3', ' FC5', ' AF3', ' F7', ' T7', ' P7', ' O1', ' O2', ' P8', ' T8',
       ' F8', ' AF4', ' FC6', ' F4']

In [ ]:
upload_and_draw("wagih3&4", False, True)

In [ ]:
classify_diff_intervals(['wagih2'], all_electrodes, filter_apply=False)